In [6]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 100

In [7]:
def get_95confidence_interval(win_rate, n, upper=False):
    win_count = n-round(n*win_rate)
    var = np.var([1 for _ in range(win_count)] + [0 for _ in range(n-win_count)], ddof=1)
    if upper == True:
        return  win_rate +1.96*np.sqrt((var/n))
    return win_rate -1.96*np.sqrt((var/n))

In [10]:
usecols = ['id', 'mode', 'stage',
       'A1-weapon', 'A1-rank', 'A2-weapon', 'A2-rank',
       'A3-weapon', 'A3-rank', 'A4-weapon', 'A4-rank',
       'B1-weapon', 'B1-rank', 'B2-weapon', 'B2-rank',
       'B3-weapon', 'B3-rank', 'B4-weapon', 'B4-rank',
       'y']
drop_list = ['A1-rank','A2-rank','A3-rank','A4-rank','B1-rank','B2-rank','B3-rank','B4-rank','mode']
weapon_list = ['A1-weapon', 'A2-weapon', 'A3-weapon', 'A4-weapon', 'B1-weapon', 'B2-weapon', 'B3-weapon', 'B4-weapon']
col_list = ['stage', 'weapon', 'win_rate', 'count']
feb_stage_list = ['gangaze', 'kombu', 'hakofugu', 'tachiuo', 'mongara', 'mozuku', 'engawa', 'anchovy']
mar_stage_list = ['battera', 'chouzame', 'fujitsubo', 'hokke', 'bbass', 'zatou', 'shottsuru', 'mutsugoro']

In [11]:
df = pd.read_csv('train_data.csv', usecols=usecols)

In [12]:
# 1年以上前のデータではあるが、2021/02現在のgame-verが 5.3.1/5.4.0 であり、大幅な update が行われていないので使ってみる
pd.read_csv('train_data.csv').head()

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,...,hokusai_becchu,NaN,26.0,herocharger_replica,NaN,68.0,sharp_neo,NaN,31.0,1
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,...,squiclean_b,NaN,118.0,campingshelter,NaN,168.0,sputtery_clear,NaN,151.0,0
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,...,nzap85,a+,163.0,prime_becchu,a-,160.0,dualsweeper_custom,a,126.0,0
3,4,2019-11-11T14:00:00+00:00,5.0.1,regular,standard,nawabari,engawa,bamboo14mk1,NaN,336,...,liter4k,NaN,189.0,promodeler_mg,NaN,194.0,hotblaster_custom,NaN,391.0,0
4,5,2019-12-14T06:00:00+00:00,5.0.1,gachi,standard,hoko,chozame,bold_7,x,299,...,sputtery_hue,x,45.0,bucketslosher_soda,x,246.0,wakaba,x,160.0,1


In [13]:
df.head()

,id,mode,stage,A1-weapon,A1-rank,A2-weapon,A2-rank,A3-weapon,A3-rank,A4-weapon,A4-rank,B1-weapon,B1-rank,B2-weapon,B2-rank,B3-weapon,B3-rank,B4-weapon,B4-rank,y
0,1,nawabari,sumeshi,sshooter_becchu,NaN,soytuber_custom,NaN,pablo_hue,NaN,hokusai,NaN,bold_7,NaN,hokusai_becchu,NaN,herocharger_replica,NaN,sharp_neo,NaN,1
1,2,nawabari,arowana,parashelter_sorella,NaN,jetsweeper,NaN,campingshelter_camo,NaN,nzap85,NaN,momiji,NaN,squiclean_b,NaN,campingshelter,NaN,sputtery_clear,NaN,0
2,3,hoko,ama,nzap89,a-,quadhopper_black,a,prime_becchu,a,jetsweeper,a,bold_7,a-,nzap85,a+,prime_becchu,a-,dualsweeper_custom,a,0
3,4,nawabari,engawa,bamboo14mk1,NaN,splatroller_becchu,NaN,dynamo_tesla,NaN,prime_becchu,NaN,splatcharger_becchu,NaN,liter4k,NaN,promodeler_mg,NaN,hotblaster_custom,NaN,0
4,5,hoko,chozame,bold_7,x,hissen_hue,x,h3reelgun_d,x,splatroller,x,furo,x,sputtery_hue,x,bucketslosher_soda,x,wakaba,x,1


In [8]:
df_area = df[df['mode']=='area']

In [9]:
def eliminate_duplicates(_df):
    for x in range(len(_df)):
        for i in range(4):
            for j in range(4):
                item = _df.iloc[x, i+3]
                if item == _df.iloc[x,j+7]:
                    _df.iloc[x,i+3] = np.nan
                    _df.iloc[x,j+7] = np.nan
                    break
    return _df

In [10]:
def modify_result(_df):
    _df = pd.melt(_df, id_vars=['id', 'stage', 'y'], value_vars=weapon_list, value_name='weapon').drop('variable', axis=1)
    _df.iloc[len(_df)//2:, 2] = _df.iloc[len(_df)//2:, 2].apply(lambda x : 1-x)
    return _df

In [11]:
df_area_a = df_area[df_area['A1-rank'].str.contains('a')].drop(drop_list, axis=1).reset_index(drop=True)
df_area_s = df_area[df_area['A1-rank'] == 's'].drop(drop_list, axis=1).reset_index(drop=True)
df_area_sp = df_area[df_area['A1-rank'] == 's+'].drop(drop_list, axis=1).reset_index(drop=True)
df_area_x = df_area[df_area['A1-rank'] == 'x'].drop(drop_list, axis=1).reset_index(drop=True)
df_area_asp = df_area[df_area['A1-rank'].str.contains('a|s')].drop(drop_list, axis=1).reset_index(drop=True)
df_area_ssp = df_area[df_area['A1-rank'].str.contains('s')].drop(drop_list, axis=1).reset_index(drop=True)
df_area_aspx = df_area[df_area['A1-rank'].str.contains('a|s|x')].drop(drop_list, axis=1).reset_index(drop=True)

In [12]:
df_area_a = eliminate_duplicates(df_area_a)
df_area_s = eliminate_duplicates(df_area_s)
df_area_sp = eliminate_duplicates(df_area_sp)
df_area_x = eliminate_duplicates(df_area_x)
df_area_asp = eliminate_duplicates(df_area_asp)
df_area_ssp = eliminate_duplicates(df_area_ssp)
df_area_aspx = eliminate_duplicates(df_area_aspx)

In [13]:
df_a = modify_result(df_area_a)
df_s = modify_result(df_area_s)
df_sp = modify_result(df_area_sp)
df_x = modify_result(df_area_x)
df_asp = modify_result(df_area_asp)
df_ssp = modify_result(df_area_ssp)
df_aspx = modify_result(df_area_aspx)

In [14]:
df_area_a = df_a.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_s = df_s.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_sp = df_sp.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_x = df_x.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_asp = df_asp.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_ssp = df_ssp.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_aspx = df_ssp.groupby(['stage','weapon']).agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_area_a.columns = ['stage', 'weapon', 'a_win_rate', 'a_count']
df_area_s.columns = ['stage', 'weapon', 's_win_rate', 's_count']
df_area_sp.columns = ['stage', 'weapon', 'sp_win_rate', 'sp_count']
df_area_x.columns = ['stage', 'weapon', 'x_win_rate', 'x_count']
df_area_asp.columns = ['stage', 'weapon', 'win_rate', 'count']
df_area_ssp.columns = ['stage', 'weapon', 'win_rate', 'count']
df_area_aspx.columns = ['stage', 'weapon', 'win_rate', 'count']

In [15]:
df_feb = df_asp[df_asp['stage'].isin(feb_stage_list)].groupby('weapon').agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_feb['min95'] = [get_95confidence_interval(i, j, False) for i, j  in zip(df_feb['y'], df_feb['id'])]

In [22]:
df_mar = df_asp[df_asp['stage'].isin(mar_stage_list)].groupby('weapon').agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)
df_mar['min95'] = [get_95confidence_interval(i, j, False) for i, j  in zip(df_mar['y'], df_mar['id'])]

In [23]:
df_mar.sort_values('min95', ascending=False)

,weapon,y,id,min95
93,prime_becchu,0.573566,802,0.539338
7,bamboo14mk3,0.750000,16,0.537824
12,bold_7,0.598802,167,0.524463
19,campingshelter,0.705882,17,0.489282
32,explosher_custom,0.593023,86,0.489192
106,screwslosher_becchu,0.570370,135,0.486865
11,bold,0.530973,339,0.477849
42,heroroller_replica,0.542056,214,0.475302
76,momiji,0.532423,293,0.475291
97,promodeler_rg,0.636364,33,0.472235


In [24]:
df_ssp[df_ssp['stage'].isin(mar_stage_list)].groupby('weapon').agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)

,weapon,y,id
93,prime_becchu,0.565079,630
83,nzap85,0.493266,594
27,dualsweeper_custom,0.505330,469
138,wakaba,0.475177,423
115,splatroller,0.494048,336
84,nzap89,0.481818,330
125,sputtery_clear,0.520492,244
11,bold,0.523404,235
76,momiji,0.509901,202
33,furo,0.374359,195


In [32]:
df_ssp.groupby('weapon').agg({'y':'mean', 'id':'count'}).reset_index().sort_values('id', ascending=False)

,weapon,y,id
93,prime_becchu,0.513017,1959
83,nzap85,0.510283,1799
27,dualsweeper_custom,0.521073,1566
138,wakaba,0.497072,1537
115,splatroller,0.504798,1042
84,nzap89,0.504399,1023
125,sputtery_clear,0.491627,836
11,bold,0.518767,746
10,barrelspinner_remix,0.517730,705
42,heroroller_replica,0.527108,664


In [769]:
df_area_x.query('weapon == "nzap85"').merge(
df_area_x.query('weapon == "nzap89"'), on='stage').merge(
df_area_x.query('weapon == "dualsweeper_custom"'), on='stage').sort_values('x_win_rate_x', ascending=False)

,stage,weapon_x,x_win_rate_x,x_count_x,weapon_y,x_win_rate_y,x_count_y,weapon,x_win_rate,x_count
20,battera,nzap85,0.687500,16,nzap89,0.000000,1,dualsweeper_custom,0.466667,15
16,manta,nzap85,0.567901,81,nzap89,0.460000,50,dualsweeper_custom,0.348837,86
13,fujitsubo,nzap85,0.563830,94,nzap89,0.533333,15,dualsweeper_custom,0.511628,86
4,sumeshi,nzap85,0.545455,154,nzap89,0.484848,33,dualsweeper_custom,0.506024,166
5,chozame,nzap85,0.539474,152,nzap89,0.413793,29,dualsweeper_custom,0.540816,98
14,kombu,nzap85,0.538462,91,nzap89,0.357143,14,dualsweeper_custom,0.533333,60
6,ama,nzap85,0.532787,122,nzap89,0.416667,24,dualsweeper_custom,0.500000,104
0,tachiuo,nzap85,0.526829,205,nzap89,0.312500,48,dualsweeper_custom,0.450704,142
1,mutsugoro,nzap85,0.524510,204,nzap89,0.363636,44,dualsweeper_custom,0.559406,202
2,hakofugu,nzap85,0.508108,185,nzap89,0.382979,47,dualsweeper_custom,0.593750,192


In [28]:
weapon = 'nzap85'
df_area_ssp.query('weapon == @weapon').sort_values('win_rate', ascending=False)

,stage,weapon,win_rate,count
897,devon,nzap85,0.622222,90
1613,kombu,nzap85,0.620000,50
1261,gangaze,nzap85,0.616667,60
1385,hakofugu,nzap85,0.567568,148
632,bbass,nzap85,0.560000,50
1141,fujitsubo,nzap85,0.547368,95
2384,tachiuo,nzap85,0.526718,131
72,ajifry,nzap85,0.521277,94
765,chozame,nzap85,0.518868,106
1505,hokke,nzap85,0.507042,71


In [31]:
weapon = 'nzap85'
df_area_a.query('weapon == @weapon').merge(
df_area_s.query('weapon == @weapon'), on=['stage', 'weapon']).merge(
df_area_sp.query('weapon == @weapon'), on=['stage', 'weapon']).merge(
df_area_x.query('weapon == @weapon'), on=['stage', 'weapon']).sort_values('a_win_rate', ascending=False)

,stage,weapon,a_win_rate,a_count,s_win_rate,s_count,sp_win_rate,sp_count,x_win_rate,x_count
20,battera,nzap85,1.000000,5,0.500000,12,0.166667,6,0.687500,16
17,hokke,nzap85,0.875000,8,0.472222,36,0.542857,35,0.495050,101
19,fujitsubo,nzap85,0.666667,6,0.464286,28,0.582090,67,0.563830,94
8,gangaze,nzap85,0.653846,26,0.517241,29,0.709677,31,0.447368,76
10,hakofugu,nzap85,0.636364,22,0.675000,40,0.527778,108,0.508108,185
16,mongara,nzap85,0.615385,13,0.538462,26,0.444444,27,0.480000,50
5,tachiuo,nzap85,0.600000,30,0.492754,69,0.564516,62,0.526829,205
9,chozame,nzap85,0.565217,23,0.482759,58,0.562500,48,0.539474,152
4,engawa,nzap85,0.548387,31,0.421053,19,0.411765,17,0.460000,100
2,bbass,nzap85,0.540541,37,0.571429,14,0.555556,36,0.427273,110


In [302]:
pd.DataFrame({
    'A' : df_a['weapon'].value_counts().head(15).index,
    'S' : df_s['weapon'].value_counts().head(15).index,
    'S+' : df_sp['weapon'].value_counts().head(15).index,
    'X' : df_x['weapon'].value_counts().head(15).index,
})

,A,S,S+,X
0,nzap89,nzap85,prime_becchu,wakaba
1,prime_becchu,prime_becchu,wakaba,prime_becchu
2,nzap85,dualsweeper_custom,dualsweeper_custom,nzap85
3,dualsweeper_custom,wakaba,nzap85,dualsweeper_custom
4,splatroller,splatroller,nzap89,kugelschreiber_hue
5,wakaba,nzap89,splatroller,l3reelgun
6,bold,bold,sputtery_clear,barrelspinner_remix
7,momiji,sputtery_clear,barrelspinner_remix,explosher_custom
8,sputtery_clear,furo,heroroller_replica,hydra_custom
9,furo,barrelspinner_remix,bold,l3reelgun_d


In [784]:
weapon = 'nzap85'
df_area_x.query('weapon == @weapon').sort_values('x_win_rate', ascending=False)

,stage,weapon,x_win_rate,x_count
590,battera,nzap85,0.687500,16
1898,manta,nzap85,0.567901,81
1234,fujitsubo,nzap85,0.563830,94
2434,sumeshi,nzap85,0.545455,154
838,chozame,nzap85,0.539474,152
1769,kombu,nzap85,0.538462,91
210,ama,nzap85,0.532787,122
2571,tachiuo,nzap85,0.526829,205
2167,mutsugoro,nzap85,0.524510,204
1496,hakofugu,nzap85,0.508108,185


In [776]:
stage = 'gangaze'
df_area_x.query('stage == @stage & x_count>20').sort_values('x_win_rate', ascending=False)

,stage,weapon,x_win_rate,x_count
1301,gangaze,bold_7,0.681818,22
1388,gangaze,sharp_neo,0.666667,21
1412,gangaze,sshooter_collabo,0.656250,32
1345,gangaze,kugelschreiber_hue,0.645833,48
1348,gangaze,l3reelgun_d,0.607143,28
1318,gangaze,furo,0.566667,30
1314,gangaze,dynamo_becchu,0.555556,45
1299,gangaze,barrelspinner_remix,0.555556,27
1415,gangaze,wakaba,0.554054,74
1375,gangaze,prime_becchu,0.547170,53


,stage,weapon,x_win_rate,x_count
2222,mutsugoro,wakaba,0.569343,274
2177,mutsugoro,prime_becchu,0.511013,227
1551,hakofugu,wakaba,0.513274,226
2571,tachiuo,nzap85,0.526829,205
2167,mutsugoro,nzap85,0.524510,204
2112,mutsugoro,dualsweeper_custom,0.559406,202
1506,hakofugu,prime_becchu,0.562814,199
1441,hakofugu,dualsweeper_custom,0.593750,192
1496,hakofugu,nzap85,0.508108,185
2355,shottsuru,wakaba,0.451087,184
